### En este calculamos KNN con la base completa (5000 datos)


In [32]:
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt

%run ../val-cruzada/pca.ipynb
%run ../knn/KNN.ipynb
%run ../datos/datos.ipynb # me importo dataset

## Validación cruzada 

Calculo PCA de cada partición y lo guardo en un txt para ahorrar tiempos

In [4]:
def particiones(x_train):
    n = len(x_train)
    alto = n // 4
    indices = np.arange(n)
    
    for i in range(4):
        train_indices = np.concatenate((indices[:i * alto], indices[(i + 1) * alto:]))

        # Hago pca con x_actual_train y reduzco componentes principales
        # en pca genero los txt
        pca(x_train[train_indices], i)
        
    return
# particiones(X_train)

Calculos del todos los pares p y k 

In [ ]:
# Función para calcular promedios
def k_fold(x_train, y_train, distancia):
    n = len(x_train)
    alto = n // 4
    indices = np.arange(n)
    
    matriz_rendimientos = np.zeros((len(intentos_p), len(intentos_k)))
    
    
    for i in range(4):
        test_indices = indices[i * alto: (i + 1) * alto]
        train_indices = np.concatenate((indices[:i * alto], indices[(i + 1) * alto:]))
        # np.random.shuffle(train_indices)
        
        autovectores_train = np.loadtxt(f'../val-cruzada/particiones/autovectores_cov{i}.txt')
        
        for p_idx, p in enumerate(intentos_p):
            x_autovectoresActual = autovectores_train[:, :p]
            
            # Centralizar los datos de entrenamiento y prueba para esta partición
            x_train_centralizado = centralizar(x_train[train_indices])
            x_test_centralizado = centralizar(x_train[test_indices])
            
            # Reducir dimensionalidad usando los autovectores seleccionados
            x_auto_red = np.dot(x_train_centralizado, x_autovectoresActual)
            x_test_red = np.dot(x_test_centralizado, x_autovectoresActual)

            for k_idx, k in enumerate(intentos_k):
                rendimiento = knn(x_auto_red, x_test_red, y_train[train_indices], y_train[test_indices], k, distancia)
                matriz_rendimientos[p_idx, k_idx] += rendimiento

    matriz_rendimientos /= 4  # Promedio a través de las 4 particiones
    return matriz_rendimientos

Nos quedamos con valor medio de p y menor k

In [7]:
def mejorPar(intentos_p, intentos_k, matriz_rendimientos):
    # Encontrar los índices del valor máximo en la matriz de rendimientos
    max_rendimiento = np.max(matriz_rendimientos)
    indices_maximos = np.argwhere(matriz_rendimientos == max_rendimiento)

    # Nos quedamos con el punto medio del rango de p
    min_p, max_p = np.min(intentos_p), np.max(intentos_p)
    punto_medio_p = (min_p + max_p) / 2
    menor_diferencia_p = float('inf')
    mejor_p = mejor_k = None

    # Selecciona el mejor par (p, k) con el p más cercano al punto medio
    for indice in indices_maximos:
        p_idx, k_idx = indice
        p, k = intentos_p[p_idx], intentos_k[k_idx]
        diferencia_p = abs(p - punto_medio_p)

        if (diferencia_p < menor_diferencia_p or (diferencia_p == menor_diferencia_p and k < mejor_k)):
            mejor_p = p
            mejor_k = k
            menor_diferencia_p = diferencia_p

    print(f"Mejor par encontrado: (p = {mejor_p}, k = {mejor_k})")
    return mejor_p, mejor_k


In [1]:
def evaluar_mejor_modelo(x_train, y_train, x_test, y_test, p, k, distancia):
    # Aplicar PCA con int(p)
    autovectores_train = np.loadtxt(f'../pca/covarianza-c/autovectores_cov.txt')
    
    x_train_pca = np.dot(x_train, autovectores_train[:, :int(p)])
    x_test_pca = np.dot(x_test, autovectores_train[:, :int(p)])
    
    # Evaluar rendimiento con kNN usando int(k)
    rendimiento = knn(x_train_pca, x_test_pca, y_train, y_test, int(k), distancia)
    return rendimiento

Para guardar los resultados en un txt

In [52]:
def guardar(rendimientos, distancia, mejor_p, mejor_k):
    mejor_p_k_file = f'../val-cruzada/resultados/mejor_p_k_{distancia}.txt'
    matriz_rendimientos_file = f'../val-cruzada/resultados/matriz_rendimientos_{distancia}.txt'

    # Borro txt viejo
    for file_path in [mejor_p_k_file, matriz_rendimientos_file]:
        if os.path.exists(file_path):
            os.remove(file_path)

    # guardo mejor_p and mejor_k
    with open(mejor_p_k_file, 'a') as f:
        np.savetxt(f, [[mejor_p, mejor_k]], fmt='%.6f')

    # guardo rendimientos
    with open(matriz_rendimientos_file, 'a') as f:
        np.savetxt(f, rendimientos, fmt='%.6f', delimiter=' ')